# Complex Net
blabla

In [1]:
# A bit of setup
from __future__ import print_function
import os,sys,inspect
import torch
import torch.optim as optim
import numpy as np
import time
import random
from scipy import fft
from torch import nn
from resampy import resample
from sklearn.metrics import average_precision_score
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from intervaltree import Interval, IntervalTree


from music.resample import resample_musicnet
from c2nn.utils import SignalDataset_music
from c2nn.model import TransformerModel,TransformerGenerationModel
from c2nn.train import train_transformer,train_model

train len 0
train len 0


# Data process


In [2]:
#resample
resample_musicnet("music/musicnet.npz", "music/musicnet_11khz.npz", 44100, 11000)

.. resampling music/musicnet.npz (44100Hz) into music/musicnet_11khz.npz (11000Hz)
.. sampling with ratio 0.2494331065759637
.. aggregating 1788 (0 / 330)
.. aggregating 1789 (1 / 330)
.. aggregating 2659 (2 / 330)
.. aggregating 2127 (3 / 330)
.. aggregating 1818 (4 / 330)
.. aggregating 1819 (5 / 330)
.. aggregating 2614 (6 / 330)
.. aggregating 2138 (7 / 330)
.. aggregating 2297 (8 / 330)
.. aggregating 2131 (9 / 330)
.. aggregating 1812 (10 / 330)
.. aggregating 1775 (11 / 330)
.. aggregating 2322 (12 / 330)
.. aggregating 1777 (13 / 330)
.. aggregating 1776 (14 / 330)
.. aggregating 1771 (15 / 330)
.. aggregating 2325 (16 / 330)
.. aggregating 1772 (17 / 330)
.. aggregating 2486 (18 / 330)
.. aggregating 2487 (19 / 330)
.. aggregating 2564 (20 / 330)
.. aggregating 2529 (21 / 330)
.. aggregating 2528 (22 / 330)
.. aggregating 2480 (23 / 330)
.. aggregating 2481 (24 / 330)
.. aggregating 2154 (25 / 330)
.. aggregating 2155 (26 / 330)
.. aggregating 2156 (27 / 330)
.. aggregating 21

In [2]:
#parse_file, get .npy files

fs = 11000            # samples/second
window_size = 4096    # fourier window size
d = 2048              # number of features
m = 128               # (USED BY DCN) number of distinct notes
stride = 512         # samples between windows
stride_test = 128            # stride in test set
k = 64            # number of window (time step) per piece
k_test = 64
data = np.load(open('music/musicnet_11khz.npz','rb'), encoding='latin1')

test_data = ['2303','2382','1819']
train_data = [f for f in data.files if f not in test_data]
index = 0

In [4]:
"""
create train data set
"""
print("len(train_data)",len(train_data))
for i in range(len(train_data)):
    print(i)
    X,Y = data[train_data[i]]
    for p in range(int((len(X)-window_size)/stride/k)):
        Xtrain = np.empty([k,d,2])
        Ytrain = np.zeros([k,m])
        for j in range(k):
            s = j*stride+p*k*stride# start from one second to give us some wiggle room for larger segments
            X_fft = fft(X[s:s+window_size])
            Xtrain[j, :, 0] = X_fft[0:d].real
            Xtrain[j, :, 1] = X_fft[0:d].imag
            # label stuff that's on in the center of the window
            for label in Y[s+d/2]:
                if (label.data[1]) >= m:
                    continue
                else:
                    Ytrain[j,label.data[1]] = 1
        Xtrain = Xtrain.reshape(k, d*2, order='F')
        np.save("music/music_train_x_64_{}.npy".format(index), Xtrain)
        np.save("music/music_train_y_64_{}.npy".format(index), Ytrain)
        index = index + 1
print("train data saved")

len(train_data) 327
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
27

In [5]:
""" 
create the test set
"""
index = 0
for i in range(len(test_data)):
    print(i)
    X,Y = data[test_data[i]]
    for p in range(int((len(X)-window_size)/stride_test/k_test)):
        Xtest = np.empty([k_test,d,2])
        Ytest = np.zeros([k_test,m])
        for j in range(k_test):
            s = j*stride_test+p*k_test*stride_test
            # start from one second to give us some wiggle room for larger segments
            X_fft = fft(X[s:s+window_size])
            Xtest[j, :, 0] = X_fft[0:d].real
            Xtest[j, :, 1] = X_fft[0:d].imag           
            # label stuff that's on in the center of the window
            for label in Y[s+d/2]:
                if (label.data[1]) >= m:
                    continue
                else:
                    Ytest[j,label.data[1]] = 1
        Xtest = Xtest.reshape(k_test, d*2, order='F')
        np.save("music/music_test_x_64_{}.npy".format(index), Xtest)
        np.save("music/music_test_y_64_{}.npy".format(index), Ytest)
        index = index + 1
print("finished")

0
1
2
finished


# Train



In [2]:
args = {
    'seed': 1111,
    'attn_dropout': 0.0, 
    'attn_mask' : False,
    'batch_size' : 16, #16
    'clip' : 0.35,
    'data' : 'music',
    'embed_dim' : 320,
    'hidden_size' : 2048,
    'lr' : 1e-4,
    'modal_lengths' : [2048, 2048],
    'model' : 'Transformer',
    'nlevels' : 6,
    'num_epochs' : 2000,
    'num_heads' : 8,
    'optim' : 'Adam',
    'out_dropout' : 0.5,
    'output_dim' : 128,
    'path' : 'music/',
    'relu_dropout' : 0.1,
    'res_dropout' : 0.1,
    'time_step' : 64
}

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

torch.manual_seed(args['seed'])
torch.cuda.manual_seed(args['seed'])
np.random.seed(args['seed'])
random.seed(args['seed'])
torch.backends.cudnn.deterministic = True


In [3]:
"""
Data Loading
"""
torch.set_default_tensor_type('torch.FloatTensor')
print("Start loading the data....")
start_time = time.time() 

training_set = SignalDataset_music(args['path'], args['time_step'], train=True)
test_set = SignalDataset_music(args['path'], args['time_step'], train=False)

print("Finish loading the data....")
train_loader = torch.utils.data.DataLoader(training_set, batch_size=args['batch_size'], shuffle=True,num_workers=16,pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=args['batch_size'], shuffle=True,num_workers=16,pin_memory=True)

"""
Train!!!
"""
#os.environ['CUDA_VISIBLE_DEVICE']='0,1,2,3'
train_transformer(args,training_set,train_loader,test_set,test_loader)

22
i 223
i 224
i 225
i 226
i 227
i 228
i 229
i 230
i 231
i 232
i 233
i 234
i 235
i 236
i 237
i 238
i 239
i 240
i 241
i 242
i 243
i 244
i 245
i 246
i 247
i 248
i 249
i 250
i 251
i 252
i 253
i 254
i 255
i 256
i 257
i 258
i 259
i 260
i 261
i 262
i 263
i 264
i 265
i 266
i 267
i 268
i 269
i 270
i 271
i 272
i 273
i 274
i 275
i 276
i 277
i 278
i 279
i 280
i 281
i 282
i 283
i 284
i 285
i 286
i 287
i 288
i 289
i 290
i 291
i 292
i 293
i 294
i 295
i 296
i 297
i 298
i 299
i 300
i 301
i 302
i 303
i 304
i 305
i 306
i 307
i 308
i 309
i 310
i 311
i 312
i 313
i 314
i 315
i 316
i 317
i 318
i 319
i 320
i 321
i 322
i 323
i 324
i 325
i 326
i 327
i 328
i 329
i 330
i 331
i 332
i 333
i 334
i 335
i 336
i 337
i 338
i 339
i 340
i 341
i 342
i 343
i 344
i 345
i 346
i 347
i 348
i 349
i 350
i 351
i 352
i 353
i 354
i 355
i 356
i 357
i 358
i 359
i 360
i 361
i 362
i 363
i 364
i 365
i 366
i 367
i 368
i 369
i 370
i 371
i 372
i 373
i 374
i 375
i 376
i 377
i 378
i 379
i 380
i 381
i 382
i 383
i 384
i 385
i 386
i 387
i 388
i

KeyboardInterrupt: 

#待办：把train的过程拆解一下，拆成model定义的过程和train的过程等